# Access PRW bounding box
src is images folders where is grouped by camera, e.g c1s1, c2s1, etc.
Read bbox.txt file

-src <br>
----c1s1 <br>
----c2s1 <br>
.
.
.

In [ ]:
import os
import re
import pandas as pd
import numpy as np
import os.path as osp
import shutil
pattern = re.compile(r"c(\d)s\d/bbox")
src = "/media/allen/mass/PRW_ReID/ClassSeries(person_id)/"
img_paths = [os.path.join(root, f) for root, _, files in os.walk(src) for f in files if ".txt" in f]

# Read bbox by pandas
Each line in bbox.txt is formated as

1. image file name
2. id
3. top left
4. left top
5. bottom right
6. right bottom 

In [ ]:
reid_info = {}
for path in img_paths:
    cam = "cam" + pattern.search(path).groups()[0]
    data = pd.read_csv(path, header=None, dtype={0:str, 1:str, 2:int, 3:int, 4:int, 5:int})
    for row in data.index:
        img, pid, x1, y1, x2, y2 = data.loc[row]
        if pid == '-2':
            continue
        if pid not in reid_info:
            reid_info[pid] = {}
            reid_info[pid][cam] = [["{}/image/{}.jpg".format(path.split("/bbox.txt")[0],img), [x1,y1,x2,y2]]]
        else:
            if cam not in reid_info[pid]:
                reid_info[pid][cam] = [["{}/image/{}.jpg".format(path.split("/bbox.txt")[0],img), [x1,y1,x2,y2]]]
            else:
                reid_info[pid][cam].append(["{}/image/{}.jpg".format(path.split("/bbox.txt")[0],img), [x1,y1,x2,y2]])

# Extract identity by camera
Making folder with identity then grouped by camera id as follow:

-$IDENTITY <br>
----cam1 <br>
&emsp;|
------images <br>
----cam2 <br>
&emsp;|
------images <br>
----cam3 <br>
&emsp;|
------images <br>
.
.
.

In [ ]:
target = np.random.choice(list(reid_info.keys()), size=10, replace=False)
for person in target:
    person_dst = osp.join("/media/allen/mass/PRW_ReID/reid_target", person)
    if not os.path.exists(person_dst):
        os.mkdir(person_dst)
    for cam in reid_info[person].keys():
        cam_dst = osp.join(person_dst, cam)
        if not os.path.exists(cam_dst):
            os.mkdir(cam_dst)
        for img_path, bbox in reid_info[person][cam]:
            shutil.copy(img_path, osp.join(cam_dst, img_path.split("/")[-1]))